In [1]:
import os
os.environ["TORCH_HOME"] = '/disks/sdb/torch_home'


%load_ext autoreload
%autoreload 2
%load_ext snoop


import itertools
import pandas
import csv
import torch
import numpy as np
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
# from allennlp.data.token_indexers.bert_tokenizers import PretrainedBertIndexer

from awesome_glue.task import Task
from awesome_glue.utils import AttackMetric
from awesome_glue.config import Config

from luna import ram_write, chunks
from allennlpx import allenutil

from luna import ram_read

from allennlpx.interpret.attackers.embedding_searcher import EmbeddingSearcher

from collections import Counter, defaultdict
from tqdm import tqdm

from copy import deepcopy
from typing import List

# pylint: disable=protected-access
import random
from collections import defaultdict
from functools import lru_cache
from itertools import product
from typing import List

import numpy as np
import torch
from allennlp.common.util import JsonDict, sanitize
from allennlp.data.fields import TextField
from allennlp.data.token_indexers import (ELMoTokenCharactersIndexer,
                                          TokenCharactersIndexer)
from allennlp.data.tokenizers import SpacyTokenizer, Token
from allennlp.modules.text_field_embedders.text_field_embedder import \
    TextFieldEmbedder
from allennlp.modules.token_embedders import Embedding

from allennlpx import allenutil
from allennlpx.interpret.attackers.attacker import (DEFAULT_IGNORE_TOKENS,
                                                    Attacker)
from allennlpx.interpret.attackers.embedding_searcher import EmbeddingSearcher
from allennlpx.interpret.attackers.policies import (CandidatePolicy,
                                                    EmbeddingPolicy,
                                                    SynonymPolicy)
from allennlpx.interpret.attackers.synonym_searcher import SynonymSearcher
from luna import cast_list, lazy_property, time_record
import copy

Loading faiss with AVX2 support.
I0301 12:59:50.624491 139623539795776 file_utils.py:38] PyTorch version 1.4.0 available.
I0301 12:59:51.843101 139623539795776 modeling.py:230] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
I0301 12:59:52.166258 139623539795776 modeling_bert.py:244] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [2]:
config = Config()
# config.finetunable=True
task = Task(config) # type: Task
task.from_pretrained()
config

cache for SST-spacy exists
*** load SST-spacy from cache cost 2.75 seconds
cache for 2f2a46e51336b27a7262689b00016955 exists
*** load 2f2a46e51336b27a7262689b00016955 from cache cost 0.0351 seconds
Restore model from checkpoint saved/models/SST-lstm-glove@0.ckpt.


Basic Args:
	--task_id=SST
	--finetunable=True
	--arch=lstm
	--pretrain=glove
	--_model_name=SST-lstm-glove
	--mode=attack
	--adv_data=nogit/SST-bert.adv.tsv
	--aug_data=
	--attack_vectors=glove
	--attack_data_split=dev
	--attack_size=100
	--attack_gen_aug=False
	--attack_gen_adv=False
	--alchemist=False
	--seed=2
Deduced Args:
	--model_name=SST-lstm-glove
	--tokenizer=spacy

In [ ]:
from allennlpx.interpret.attackers.pwws import PWWS
attacker = PWWS(task.predictor)
attacker.initialize()
sent = allenutil.as_sentence(task.dev_data[15])
print(sent)
print(task.predictor.predict(sent), '\n')
result = attacker.attack_from_json({"sent": sent},
                                   field_to_change="sent",
#                                    ignore_tokens=forbidden_words,
#                                    forbidden_tokens=forbidden_words,
                                   max_change_num_or_ratio=0.25,
                                   measure='euc',
                                   topk=300,
#                                                    search_num=256
                                  )
print(result)
print(allenutil.as_sentence(result['raw']))
print(allenutil.as_sentence(result['adv']))

In [ ]:
import torch

# List available models
torch.hub.list('pytorch/fairseq')  # [..., 'transformer_lm.wmt19.en', ...]

# Load an English LM trained on WMT'19 News Crawl data
en_lm = torch.hub.load('pytorch/fairseq', 'transformer_lm.wmt19.en', tokenizer='moses', bpe='fastbpe')
en_lm.eval()  # disable dropout

# Move model to GPU
en_lm.cuda(1)

# Sample from the language model
en_lm.sample('Barack Obama', beam=1, sampling=True, sampling_topk=10, temperature=0.8)
# "Barack Obama is coming to Sydney and New Zealand (...)"

# Compute perplexity for a sequence
en_lm.score('Barack Obama is coming to Sydney and New Zealand')['positional_scores'].mean().neg().exp()



In [228]:
lst = ['Barack Obama is coming to Beijing'] * 4
results = en_lm.score(lst)
ppls = [ele['positional_scores'].mean().neg().exp() for ele in results]
ppls

[tensor(101.9296, device='cuda:1'),
 tensor(101.9296, device='cuda:1'),
 tensor(101.9296, device='cuda:1'),
 tensor(101.9296, device='cuda:1')]

In [ ]:
help(en_lm)

In [25]:
kwargs = { 
#                 "ignore_tokens": ["is"],
#                 "forbidden_tokens": ["is"],
                "max_change_num_or_ratio": 0.25
            }
attacker = Genetic(task.predictor, **kwargs)
attacker.initialize()
attacker.perturb("the dog is lovely".split(" "),
                                "the cat is lovely".split(" "),)
result = attacker.attack_from_json({'sent': allenutil.as_sentence(task.dev_data[2])},
                         field_to_change='sent')
if result['success'] == 1:
    print(allenutil.as_sentence(result['raw']))
    print(allenutil.as_sentence(result['adv']))
    print(result['outputs'])
    print(result['success'])
    print(result['generation'])

Using cache found in /disks/sdb/torch_home/hub/pytorch_fairseq_master
I0301 13:09:20.997089 139623539795776 file_utils.py:71] loading archive file https://dl.fbaipublicfiles.com/fairseq/models/lm/wmt19.en.tar.bz2 from cache at /disks/sdb/torch_home/pytorch_fairseq/0cabe4a3e5ef7de13b205be3ab7feaf000768c332991cab28503a5464cbd5133.1194dafe66fa251b0e81fb532efac27216c501bfb0573c5d7e77e04dd6d88b15
I0301 13:09:23.297399 139623539795776 language_modeling.py:116] dictionary: 42022 types
I0301 13:09:32.551391 139623539795776 fairseq_model.py:198] Namespace(activation_fn='relu', adaptive_input=False, adaptive_input_cutoff=None, adaptive_input_factor=4, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, adaptive_softmax_factor=4, add_bos_token=False, arch='transformer_lm_gbw', attention_dropout=0.1, bpe='fastbpe', bpe_codes='/disks/sdb/torch_home/pytorch_fairseq/0cabe4a3e5ef7de13b205be3ab7feaf000768c332991cab28503a5464cbd5133.1194dafe66fa251b0e81fb532efac27216c501bfb0573c5d7e77e04dd6d88b15/

the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales .
the acting , costumes , music , cinematography and noise are all impressive given the project one bleak locales .
{'logits': [0.13473094999790192, -0.17793287336826324], 'probs': [0.5775353312492371, 0.42246463894844055]}
1
5


In [26]:
result = attacker.attack_from_json({'sent': allenutil.as_sentence(task.dev_data[2])},
                         field_to_change='sent')
if result['success'] == 1:
    print(allenutil.as_sentence(result['raw']))
    print(allenutil.as_sentence(result['adv']))
    print(result['outputs'])
    print(result['success'])
    print(result['generation'])

the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales .
the acting , costumes , song , cinematography and noise are all impressive given the production 's desolate locales .
{'logits': [0.0036784783005714417, -0.028392136096954346], 'probs': [0.5080170035362244, 0.491983026266098]}
1
4


In [34]:
class A:
    def __init__(self, m):
        self.m = m
        
class B(A):
    def __init__(self, c, *, d, **kwargs):
        super().__init__(**kwargs)
        print(kwargs)
        self.c = c
        self.d = d
        
b = B(c=1, d=2, m=3)
print(b.c)
print(b.m)

{'m': 3}
1
3
